# TEXT MINING for PRACTICE: 동적 페이지 수집 using Selenium
---

## 1. Selenium 패키지 다운로드 및 설치

In [1]:
import sys
!{sys.executable} -m pip install selenium

- 크롬 버전 확인 후 알맞은 드라이버 다운로드
- 주소창에 chrome://settings/help 입력
- http://chromedriver.chromium.org/downloads에 접속하여 다운로드

## 2. 관련 패키지 Import

In [72]:
# Python 코드를 통해 가상의 브라우저를 띄우기 위해 selenium 패키지를 import 합니다.
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

# selenium을 활용해 브라우저를 직접 띄우는 경우, 실제 웹서핑을 할때처럼 로딩시간이 필요합니다.
# 로딩시간 동안 대기하도록 코드를 구성하기위해 time 패키지를 import 합니다.
import time

# Python 코드를 통해 웹페이지에 정보를 요청하기 위해 BeautifulSoup, urllib 패키지를 import 합니다.
from bs4 import BeautifulSoup
import requests
import urllib

## 3. 크롬 브라우저 열기

In [73]:
# Chrome Driver를 호출합니다.
chrome_options = webdriver.ChromeOptions()

# 브라우저에 임의로 User-agent 옵션을 넣어 Python 코드로 접속함을 숨깁니다.
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')

# Chrome Driver 파일의 경로를 지정하고 실행합니다.
# Chrome Driver는 아래 링크에서 다운로드 가능합니다.

# 본 Jupyter Notebook 파일과 동일한 경로에 Chrome Driver가 존재하는 경우 아래 경로를 그대로 사용합니다.

# Windows 운영체제
#driver = webdriver.Chrome(executable_path = "chromedriver", chrome_options=chrome_options)

# MAC, Linux 운영체제
driver = webdriver.Chrome(executable_path = "./chromedriver", chrome_options=chrome_options)

/Users/seojungwon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


## 4. 인스타그램 접속

In [74]:
# 브라우저에서 URL에 해당하는 페이지로 이동합니다.
URL = "https://www.instagram.com/"
driver.get(URL)
# 실제 페이지가 불러와지는 시간을 고려해 sleep(SEC) 함수로 기다리는 시간을 지정해줍니다.
time.sleep(5)

## 5. 로그인 하기

In [75]:
# 인스타그램 게시물에 접근하기 위해서는 로그인이 필요합니다.
# 인스타그램 로그인 페이지로 이동합니다.
# click() 함수로 원하는 요소(태그)를 클릭할 수 있습니다.
driver.find_element(By.CLASS_NAME, "izU2O").find_element(By.TAG_NAME, "a").click()
time.sleep(5)

In [76]:
# 인스타그램 로그인 페이지에서 아이디(ID)와 비밀번호(PW)를 입력합니다.
# 아이디와 비밀번호를 브라우저에서 직접 입력해도 됩니다.
ID = "YOUR_ID"
PW = "YOUR_PASSWORD"
driver.find_element(By.NAME, "username").send_keys(ID)
driver.find_element(By.NAME, "password").send_keys(PW)
time.sleep(2)

In [77]:
driver.find_element(By.CLASS_NAME, "L3NKy").click()
time.sleep(5)

In [78]:
driver.find_element(By.CLASS_NAME, "mt3GC").find_elements(By.TAG_NAME, "button")[1].click()
time.sleep(5)

## 6. 검색어 입력

In [79]:
keyword = "#손흥민"
search_form = driver.find_element(By.CSS_SELECTOR, "#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.LWmhU._0aCwM > input")
search_form.send_keys(keyword)
time.sleep(5)

In [80]:
search_url = driver.find_element(By.CLASS_NAME, "fuqBx").find_element(By.TAG_NAME, "a").get_attribute("href")
driver.get(search_url)
time.sleep(5)

## 7. 검색 결과 로딩 후 수집 시작!

In [81]:
# 수집된 게시물 개수를 저장할 변수를 선언합니다.
post_count = 0

# 게시물 내용을 저장할 파일을 생성합니다.
# 파일명에는 "#"을 포함할 수 없으므로 제거해줍니다.
f = open("instagram_post_" + keyword.replace("#", "") + ".txt", "w", encoding = "utf-8")

# 인스타그램 게시물은 스크롤을 계속 내려야 모든 게시물을 확인할 수 있습니다.
# 몇 번 스크롤을 내리면서 게시물을 수집할지 지정합니다.
# 게시물이 양이 많을수록 스크롤 횟수를 늘려줍니다.
scroll_limit = 2

# 수집된 전체 게시물을 저장할 리스트를 생성합니다.
# 중복된 게시물 수집을 방지하기위해 활용됩니다.
total_post_list = []

for i in range(scroll_limit+1):
    # 현재 내려진 스크롤 횟수를 출력합니다.
    print("Scroll 횟수 :", i, end="\r")
    time.sleep(5)
    # 현재 화면상에 출현한 게시물 모두를 불러와 리스트에 저장합니다.
    post_list = driver.find_elements(By.CLASS_NAME, "_9AhH0")
    # 이미 수집된 게시물이 있는지 확인하고 새로운 게시물만 리스트에 저장합니다.
    post_list = [post for post in post_list if post not in total_post_list]
    # 수집할 게시물을 수집된 전체 게시물 리스트에 추가합니다.
    total_post_list += post_list
    # FOR 문을 활용해 게시물 지정을 반복합니다. (1.~4. 반복)
    for post in post_list:
        try:
            # 1. 포스팅 이미지를 클릭합니다.
            post.click()
            time.sleep(7)
            # 2. 포스팅 본문의 텍스트를 가져옵니다.
            try:
                content = driver.find_element(By.CLASS_NAME, "C4VMK").text
                content = content.replace("\n", " ")
                print(content)
                f.write(content + "\n")
            except:
                None
            # 3. 수집한 포스트 개수를 늘려줍니다.
            post_count += 1
            # 4. 우측 상단의 X를 눌러 팝업창을 닫습니다.
            driver.find_element(By.CLASS_NAME, "ckWGn").click()
        except:
            continue
    time.sleep(5)        
    # 스크롤을 내려 추가로 게시물이 브라우저에 보이도록 합니다.
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
f.close()

hm_banana * 볼리비아전 편집하다가 ( ⁼̴̤̆◡̶͂⁼̴̤̆ ) 입꼬리 진짜💙 . 𖤐 𝘴𝘰𝘶𝘳𝘤𝘦 : kbs2 𖤐 𝘦𝘥𝘪𝘵 : @𝘩𝘮_𝘣𝘢𝘯𝘢𝘯𝘢 - - - #손흥민 #sonheungmin #heungminson #sonny #HM7 #국가대표손흥민🇰🇷 #대한민국vs볼리비아 #친선경기 #손흥민사랑함듕💙 17시간
huieon94 #손흥민 시즌 첫 골과 멀티골 넣은 후 모습 입니다! 오늘은 정말 손흥민의 날이네요. 토트넘 4-0 크리스탈 팰리스 with @goal_korea #축스타그램 #일상 #데일리 #축구 #취재 #프리미어리그 #EPL #현장 #골닷컴 #토트넘 #크팰 #경기 #영상 #직촬 #instargood #football #tottenhamhotspur #match #sonheungmin #goal #son #good #⚽ 10시간
hm__son7__ 아아아ㅠㅠ이렇게 웃으면ㅠㅠ사람 죽어요😭😭❤️ 출처 @goal_korea #손흥민 1시간
hm7__son 뭔일이야ㅠㅠㅠㅠㅠㅠㅠㅠ 수정궁 상대로 A매치 끝나고 와서 이렇게 강하네 진짜ㅠㅠㅠㅠㅠㅠㅠㅜㅠ 골 기대 많이 안하고 있었는데 짜릿하다 초반부터💙💙💙💙 영상 올리려고 하고있는데 와중에 멀티골 실화냐,,,,,,,, ⠀⠀⠀⠀⠀⠀⠀⠀⠀ OMG SONNY'S SEASON FIRST GOAL🔥🔥🔥🔥AND HE JUST SCORED HIS SECOND GOAL WHEN I WAS UPLOADING THIS VIDEO OMG🤣🤣🤣😍😍😍 ⠀⠀⠀⠀⠀⠀⠀⠀⠀ #손흥민 #COYS 14시간
kimjejelly 우리흥 시소코랑 핸드셰이크 🙌🏻하고... 토트넘 선수님들도 수고하셨네요👏🏻👏🏻👏🏻 #손흥민#빛흥민#우리흥#손세이셔널#sonheungmin#토트넘#tottenhamhotspur#SON7#🌟#⚽️ 출처 이건의발품스토리tv 🙏🏻 7시간
serendipity_097 #토트넘 vs #크리스탈팰리스 #손흥민 #내선수손흥민 #토비 시즌 1호골🎉🎉 최고👍👍👍👍 15시간
sonny7__ 행복해보여서 나도 행복해 ㅠㅠㅠㅠㅠㅠㅠㅠㅠ💙 1

In [82]:
print("> 수집한 총 게시물 개수 :", post_count)


> 수집한 총 게시물 개수 : 15


In [83]:
# Chrome Driver를 닫습니다.
driver.close()

In [86]:
# 파일에 저장된 카페 게시글 내용을 확인합니다.
f = open("instagram_post_" + keyword.replace("#", "") + ".txt", encoding="utf-8")
for post in f.read().split("\n")[:10]:
    print(post.strip())
f.close()


hm_banana * 볼리비아전 편집하다가 ( ⁼̴̤̆◡̶͂⁼̴̤̆ ) 입꼬리 진짜💙 . 𖤐 𝘴𝘰𝘶𝘳𝘤𝘦 : kbs2 𖤐 𝘦𝘥𝘪𝘵 : @𝘩𝘮_𝘣𝘢𝘯𝘢𝘯𝘢 - - - #손흥민 #sonheungmin #heungminson #sonny #HM7 #국가대표손흥민🇰🇷 #대한민국vs볼리비아 #친선경기 #손흥민사랑함듕💙 17시간
huieon94 #손흥민 시즌 첫 골과 멀티골 넣은 후 모습 입니다! 오늘은 정말 손흥민의 날이네요. 토트넘 4-0 크리스탈 팰리스 with @goal_korea #축스타그램 #일상 #데일리 #축구 #취재 #프리미어리그 #EPL #현장 #골닷컴 #토트넘 #크팰 #경기 #영상 #직촬 #instargood #football #tottenhamhotspur #match #sonheungmin #goal #son #good #⚽ 10시간
hm__son7__ 아아아ㅠㅠ이렇게 웃으면ㅠㅠ사람 죽어요😭😭❤️ 출처 @goal_korea #손흥민 1시간
hm7__son 뭔일이야ㅠㅠㅠㅠㅠㅠㅠㅠ 수정궁 상대로 A매치 끝나고 와서 이렇게 강하네 진짜ㅠㅠㅠㅠㅠㅠㅠㅜㅠ 골 기대 많이 안하고 있었는데 짜릿하다 초반부터💙💙💙💙 영상 올리려고 하고있는데 와중에 멀티골 실화냐,,,,,,,, ⠀⠀⠀⠀⠀⠀⠀⠀⠀ OMG SONNY'S SEASON FIRST GOAL🔥🔥🔥🔥AND HE JUST SCORED HIS SECOND GOAL WHEN I WAS UPLOADING THIS VIDEO OMG🤣🤣🤣😍😍😍 ⠀⠀⠀⠀⠀⠀⠀⠀⠀ #손흥민 #COYS 14시간
kimjejelly 우리흥 시소코랑 핸드셰이크 🙌🏻하고... 토트넘 선수님들도 수고하셨네요👏🏻👏🏻👏🏻 #손흥민#빛흥민#우리흥#손세이셔널#sonheungmin#토트넘#tottenhamhotspur#SON7#🌟#⚽️ 출처 이건의발품스토리tv 🙏🏻 7시간
serendipity_097 #토트넘 vs #크리스탈팰리스 #손흥민 #내선수손흥민 #토비 시즌 1호골🎉🎉 최고👍👍👍👍 15시간
sonny7__ 행복해보여서 나도 행복해 ㅠㅠㅠㅠㅠㅠㅠㅠㅠ💙 1

In [ ]:
#Q. 단순 Request를 이용해서는 수집 할 수 없을까?